In [2]:
import numpy as np
import sympy as sp

## Symbolic differentiation

- 일변량 함수

In [3]:
x = sp.Symbol('x')
f = sp.exp(x-2) - x
df = f.diff(x)

print(df)

exp(x - 2) - 1


In [4]:
derv = sp.lambdify(x, df, 'numpy') # x, df: symbol

# 도함수값 출력
print(derv(1))
print(derv(2))
print(derv(3))

-0.6321205588285577
0.0
1.718281828459045


- 다변량 함수

In [5]:
def f(x):
    y = x[0]**2 + x[0]*x[1] +x[1]**2
    return y

In [9]:
x = sp.IndexedBase('x')

In [10]:
sp.diff(f(x), x[0])

2*x[0] + x[1]

In [11]:
sp.diff(f(x), x[1])

x[0] + 2*x[1]

In [12]:
[sp.diff(f(x), x[i]) for i in range(2)]

[2*x[0] + x[1], x[0] + 2*x[1]]

In [13]:
# 편미분한 값들의 배열 생성
gradients = np.array([sp.diff(f(x), x[i]) for i in range(2)]) 
# 도함수 값을 구하는 함수
grads = sp.lambdify(x, gradients, 'numpy')

x_ = [1, 2]
print(grads(x_))

[4, 5]


In [14]:
def f(x):
    return sp.sin(x[0]*x[1]) + sp.exp(x[1]+x[2]) - x[2]

In [16]:
x = sp.IndexedBase('x')
# 편미분한 값들의 배열 생성
gradients = np.array([sp.diff(f(x), x[i]) for i in range(3)])
# 도함수값을 구하는 함수
grads = sp.lambdify(x, gradients, 'numpy')

x_ = [1, 2, 3]
print(grads(x_))

[-0.8322936730942848, 147.99701226602946, 147.4131591025766]


### backtracking line search

In [22]:
def backtracking_line_search(f, grads, x, d, alpha=10, p=0.5, beta=1e-4):

    y, g = f(x), grads(x)
    
    i = 1
    while f(x + alpha*d) > y + beta*alpha*np.dot(g, d): # g.T @ d
        
        alpha *= p
        print(f'{i}: alpha = {alpha:.4f}')
        
        i += 1

    return alpha

In [18]:
def f(x):
    y = x[0]**2 + x[0]*x[1] +x[1]**2
    return y

In [23]:
x = sp.IndexedBase('x')
gradients = np.array([sp.diff(f(x), x[i]) for i in range(2)])
grads = sp.lambdify(x, gradients, 'numpy')

x_ = np.array([1, 2])
d_ = np.array([-1, -1])

alpha = backtracking_line_search(f, grads, x_, d_)
print(alpha)

1: alpha = 5.0000
2: alpha = 2.5000
2.5


### Local descent algorithm
```
1. x(k) 종료조건 확인
2. descent direction d(k) 결정
3. alpha(k) 결정
4. x(k+1) = x(k) + alpha(k)*d(k)
```

In [30]:
# x = x + alpha*(g*d) : d = [-1]

x_ = np.array([1, 2])
d_ = -1 * np.array(grads(x_))

alpha = backtracking_line_search(f, grads, x_, d_)

print(alpha, x_, f(x_))

1: alpha = 5.0000
2: alpha = 2.5000
3: alpha = 1.2500
4: alpha = 0.6250
0.625 [1 2] 7


#### Termination conditions
```
1. abstol: abs(y - y_prev) < 1E-8
2. reltol: abs(y - y_prev) < 1E-8 * (abs(y_prev) + 1E-8)
3. maximum iteration
```

In [31]:
x_ = np.array([1, 2])
d_ = -1 * np.array(grads(x_))

alpha = backtracking_line_search(f, grads, x_, d_)

print(alpha, x_, f(x_))

y_prev = f(x_)
flag = True

i =1
while flag:
    x_ = x_ + alpha*d_
    d_ = -1 * np.array(grads(x_))
    
    alpha = backtracking_line_search(f, grads, x_, d_)
    
    y_ = f(x_)
    
    diff = np.abs(y_ - y_prev)
    print(i, alpha, x_, f(x_), diff)
    
    ### abstol
    if diff < 1e-8:
        flag = False
    
    y_prev = y_
    
    i += 1

1: alpha = 5.0000
2: alpha = 2.5000
3: alpha = 1.2500
4: alpha = 0.6250
1 0.625 [-1.5   -1.125] 5.203125 1.796875
1: alpha = 5.0000
2: alpha = 2.5000
3: alpha = 1.2500
4: alpha = 0.6250
2 0.625 [1.078125 1.21875 ] 3.961669921875 1.241455078125
1: alpha = 5.0000
2: alpha = 2.5000
3: alpha = 1.2500
4: alpha = 0.6250
3 0.625 [-1.03125    -0.97851562] 3.0300636291503906 0.9316062927246094
1: alpha = 5.0000
2: alpha = 2.5000
3: alpha = 1.2500
4: alpha = 0.6250
4 0.625 [0.86938477 0.88916016] 2.3194579482078552 0.7106056809425354
1: alpha = 5.0000
2: alpha = 2.5000
3: alpha = 1.2500
4: alpha = 0.6250
5 0.625 [-0.77307129 -0.76565552] 1.7757738875225186 0.5436840606853366
1: alpha = 5.0000
2: alpha = 2.5000
3: alpha = 1.2500
4: alpha = 0.6250
6 0.625 [0.67180252 0.67458344] 1.3595682898740051 0.4162055976485135
1: alpha = 5.0000
2: alpha = 2.5000
3: alpha = 1.2500
4: alpha = 0.6250
7 0.625 [-0.58956528 -0.58852243] 1.0409182635778507 0.31865002629615446
1: alpha = 5.0000
2: alpha = 2.5000
3: 

In [34]:
x_ = np.array([1, 2])
d_ = -1 * np.array(grads(x_))

alpha = backtracking_line_search(f, grads, x_, d_)

print(alpha, x_, f(x_))

y_prev = f(x_)
flag = True

i =1
while flag:
    x_ = x_ + alpha*d_
    d_ = -1 * np.array(grads(x_))
    
    alpha = backtracking_line_search(f, grads, x_, d_)
    
    y_ = f(x_)
    
    diff = np.abs(y_ - y_prev)
    print(i, alpha, x_, f(x_), diff)
    
    ### reltol
    if diff < 1e-8 * (np.abs(y_prev)+1e-8):
        flag = False
    
    y_prev = y_
    
    i += 1

1: alpha = 5.0000
2: alpha = 2.5000
3: alpha = 1.2500
4: alpha = 0.6250
0.625 [1 2] 7
1: alpha = 5.0000
2: alpha = 2.5000
3: alpha = 1.2500
4: alpha = 0.6250
1 0.625 [-1.5   -1.125] 5.203125 1.796875
1: alpha = 5.0000
2: alpha = 2.5000
3: alpha = 1.2500
4: alpha = 0.6250
2 0.625 [1.078125 1.21875 ] 3.961669921875 1.241455078125
1: alpha = 5.0000
2: alpha = 2.5000
3: alpha = 1.2500
4: alpha = 0.6250
3 0.625 [-1.03125    -0.97851562] 3.0300636291503906 0.9316062927246094
1: alpha = 5.0000
2: alpha = 2.5000
3: alpha = 1.2500
4: alpha = 0.6250
4 0.625 [0.86938477 0.88916016] 2.3194579482078552 0.7106056809425354
1: alpha = 5.0000
2: alpha = 2.5000
3: alpha = 1.2500
4: alpha = 0.6250
5 0.625 [-0.77307129 -0.76565552] 1.7757738875225186 0.5436840606853366
1: alpha = 5.0000
2: alpha = 2.5000
3: alpha = 1.2500
4: alpha = 0.6250
6 0.625 [0.67180252 0.67458344] 1.3595682898740051 0.4162055976485135
1: alpha = 5.0000
2: alpha = 2.5000
3: alpha = 1.2500
4: alpha = 0.6250
7 0.625 [-0.58956528 -0.58

In [36]:
x_ = np.array([1, 2])
d_ = -1 * np.array(grads(x_))

alpha = backtracking_line_search(f, grads, x_, d_)

print(alpha, x_, f(x_))

y_prev = f(x_)
max_iter = 100
flag = True

i =1
while flag:
    x_ = x_ + alpha*d_
    d_ = -1 * np.array(grads(x_))
    
    alpha = backtracking_line_search(f, grads, x_, d_)
    
    y_ = f(x_)
    
    diff = np.abs(y_ - y_prev)
    print(i, alpha, x_, f(x_), diff)
    
    ### maximum iteration
    if i > max_iter:
        flag = False
    
    y_prev = y_
    
    i += 1

1: alpha = 5.0000
2: alpha = 2.5000
3: alpha = 1.2500
4: alpha = 0.6250
0.625 [1 2] 7
1: alpha = 5.0000
2: alpha = 2.5000
3: alpha = 1.2500
4: alpha = 0.6250
1 0.625 [-1.5   -1.125] 5.203125 1.796875
1: alpha = 5.0000
2: alpha = 2.5000
3: alpha = 1.2500
4: alpha = 0.6250
2 0.625 [1.078125 1.21875 ] 3.961669921875 1.241455078125
1: alpha = 5.0000
2: alpha = 2.5000
3: alpha = 1.2500
4: alpha = 0.6250
3 0.625 [-1.03125    -0.97851562] 3.0300636291503906 0.9316062927246094
1: alpha = 5.0000
2: alpha = 2.5000
3: alpha = 1.2500
4: alpha = 0.6250
4 0.625 [0.86938477 0.88916016] 2.3194579482078552 0.7106056809425354
1: alpha = 5.0000
2: alpha = 2.5000
3: alpha = 1.2500
4: alpha = 0.6250
5 0.625 [-0.77307129 -0.76565552] 1.7757738875225186 0.5436840606853366
1: alpha = 5.0000
2: alpha = 2.5000
3: alpha = 1.2500
4: alpha = 0.6250
6 0.625 [0.67180252 0.67458344] 1.3595682898740051 0.4162055976485135
1: alpha = 5.0000
2: alpha = 2.5000
3: alpha = 1.2500
4: alpha = 0.6250
7 0.625 [-0.58956528 -0.58

In [39]:
def local_descent_backtracking(f, grads, x_, alpha=10, TOL=1e-8):
    
    d_ = -1 * np.array(grads(x_))
    alpha = backtracking_line_search(f, grads, x_, d_, alpha)
    print(alpha, x_, f(x_))
    
    y_prev = f(x_)
    
    i = 1
    flag = True
    
    while flag:
        x_ = x_ + alpha*d_
        d_ = -1 * np.array(grads(x_))
        
        alpha = backtracking_line_search(f, grads, x_, d_, alpha)
        
        y_ = f(x_)
        
        diff = np.abs(y_ - y_prev)
        print(i, alpha, x_, f(x_), diff)
        
        if diff < TOL * (abs(y_prev) + TOL):
            flag = False
        
        y_prev = y_
        
        i += 1
        
    return i, x_

In [40]:
x_ = np.array([1, 2])
local_descent_backtracking(f, grads, x_)

1: alpha = 5.0000
2: alpha = 2.5000
3: alpha = 1.2500
4: alpha = 0.6250
0.625 [1 2] 7
1 0.625 [-1.5   -1.125] 5.203125 1.796875
2 0.625 [1.078125 1.21875 ] 3.961669921875 1.241455078125
3 0.625 [-1.03125    -0.97851562] 3.0300636291503906 0.9316062927246094
4 0.625 [0.86938477 0.88916016] 2.3194579482078552 0.7106056809425354
5 0.625 [-0.77307129 -0.76565552] 1.7757738875225186 0.5436840606853366
6 0.625 [0.67180252 0.67458344] 1.3595682898740051 0.4162055976485135
7 0.625 [-0.58956528 -0.58852243] 1.0409182635778507 0.31865002629615446
8 0.625 [0.51521784 0.51560891] 0.796952875626598 0.24396538795125267
9 0.625 [-0.45106003 -0.45091338] 0.6101670215058836 0.18678585412071436
10 0.625 [0.39458587 0.39464086] 0.46715912248010516 0.14300789902577848
11 0.625 [-0.34529701 -0.34527638] 0.3576687026762831 0.10949041980382207
12 0.625 [0.30212199 0.30212972] 0.2738401004200772 0.08382860225620586
13 0.625 [-0.26436157 -0.26435867] 0.20965882687477683 0.0641812735453004
14 0.625 [0.23131457 

(142, array([-9.98221076e-09, -9.98221076e-09]))

### Strong backtracking with Symbolic differentitation

In [41]:
def strong_backtracking(f, grads, x, d, alpha=1.0, beta=1e-4, sigma=1e-1):
    
    y0, g0, y_prev, alpha_prev = f(x), np.dot(grads(x), d), np.nan, 0.0
    alpha_lo, alpha_hi = np.nan, np.nan
    
    while True:
        y = f(x + alpha*d)
        
        if y > y0 + beta*alpha*g0 or (not(np.isnan(y_prev)) and y >= y_prev):
            alpha_lo, alpha_hi = alpha_prev, alpha
            break
            
        g = np.dot(grads(x + alpha*d), d)
        
        if np.abs(g) <= -sigma * g0:
            return alpha
        elif g >= 0:
            alpha_lo, alpha_hi = alpha, alpha_prev
            break
            
        y_prev, alpha_prev, alpha = y, alpha, 2*alpha
    
    y_lo = f(x + alpha_lo*d)
    
    while True:
        
        alpha = 0.5 * (alpha_lo+alpha_hi)
        y = f(x + alpha*d)
        
        if (y > y0 + beta*alpha*g0) or (y >= y_lo):
            alpha_hi = alpha
        else:
            g = np.dot(grads(x + alpha*d), d)
            
            if abs(g) <= -sigma * g0:
                return alpha
            elif g * (alpha_hi-alpha_lo) >= 0.0:
                alpha_hi = alpha_lo
            
            alpha_lo = alpha

In [48]:
def local_descent_strong_backtracking(f, grads, x_, alpha=10, TOL=1e-8):
    
    d_ = -1 * np.array(grads(x_))
    alpha = strong_backtracking(f, grads, x_, d_, alpha)
    print(alpha, x_, f(x_))
    
    y_prev = f(x_)
    
    i = 1
    flag = True
    
    while flag:
        x_ = x_ + alpha*d_
        d_ = -1 * np.array(grads(x_))
        
        alpha = strong_backtracking(f, grads, x_, d_, alpha)
        
        y_ = f(x_)
        diff = np.abs(y_ - y_prev)
        print(i, alpha, x_, f(x_), diff)
        
        # reltol
        if diff < TOL * (abs(y_prev)+TOL):
            flag = False
        
        y_prev = y_
        i += 1
        
    return i, x_

In [49]:
x_ = np.array([1, 2])
local_descent_strong_backtracking(f, grads, x_)

0.3125 [1 2] 7
1 0.546875 [-0.25    0.4375] 0.14453125 6.85546875
2 0.478515625 [-0.21582031  0.09570312] 0.03508281707763672 0.10944843292236328
3 0.478515625 [-0.05506897  0.10738564] 0.008650649808259914 0.026432167269376805
4 0.59814453125 [-0.05375195  0.03096559] 0.002183678979532186 0.006466970828727728
5 0.4486083984375 [-0.00797098  0.02607323] 0.0005355205836490411 0.0016481583958831447
6 0.5046844482421875 [-0.01251595  0.00625574] 0.00011748679064494113 0.00041803379300410005
7 0.5046844482421875 [-0.00303991  0.006258  ] 2.9379829503411073e-05 8.810696114153005e-05
8 0.5046844482421875 [-0.00312983  0.00147557] 7.354874670857731e-06 2.202495483255334e-05
9 0.5046844482421875 [-0.00071537  0.00156575] 1.8432452130541135e-06 5.511629457803618e-06
10 0.5046844482421875 [-0.00078351  0.00034637] 4.6247503339690733e-07 1.3807701796572062e-06
11 0.44159889221191406 [-0.00016747  0.00039218] 1.1617311699138711e-07 3.4630191640552024e-07
12 0.6623983383178711 [-0.00019275  0.00011

(27, array([-4.46595927e-09,  2.41941620e-09]))

In [50]:
# Rosenbrock
def f(x, a=1, b=5):
    y = (a - x[0])**2 + b*(x[1] - x[0]**2)**2
    return y

In [53]:
x = sp.IndexedBase('x')
gradients = np.array([sp.diff(f(x), x[i]) for i in range(2)])
grads = sp.lambdify(x, gradients, 'numpy')

x_ = np.array([-2, 2])

In [54]:
local_descent_backtracking(f, grads, x_, alpha=1)

1: alpha = 0.5000
2: alpha = 0.2500
3: alpha = 0.1250
4: alpha = 0.0625
5: alpha = 0.0312
0.03125 [-2  2] 29
1 0.03125 [0.6875 2.625 ] 23.260574340820312 5.7394256591796875
1: alpha = 0.0156
2 0.015625 [1.63186646 1.95239258] 2.9239853889447027 20.33658895187561
3 0.015625 [1.24974655 2.06342313] 1.320168881254962 1.6038165076897408
4 0.015625 [1.43782283 1.9850549 ] 0.22553848282746347 1.0946303984274985
5 0.015625 [1.38717103 1.99791109] 0.17703599549942528 0.04850248732803819
6 0.015625 [1.40700618 1.98640052] 0.16588077392404574 0.011155221575379543
7 0.015625 [1.39724816 1.98534831] 0.16326625561522218 0.00261451830882356
8 0.015625 [1.39926331 1.98018489] 0.16188585521568646 0.001380400399535714
9 0.015625 [1.39651431 1.97670879] 0.16072334882132844 0.001162506394358026
10 0.015625 [1.39566917 1.97257495] 0.15960022536996402 0.0011231234513644217
11 0.015625 [1.3940697  1.96871831] 0.1584883378966662 0.0011118874732978234
12 0.015625 [1.39277165 1.96476706] 0.1573831273800901 0.0

281 0.015625 [1.12284148 1.26994776] 0.0155109111126746 0.0001571645139663968
282 0.015625 [1.122222  1.2685142] 0.015355182839497002 0.00015572827317759767
283 0.015625 [1.1216051  1.26708733] 0.01520087961911431 0.000154303220382691
284 0.015625 [1.12099076 1.26566712] 0.01504799032556261 0.0001528892935517001
285 0.015625 [1.12037899 1.26425355] 0.01489650389487135 0.00015148643069125972
286 0.015625 [1.11976978 1.26284661] 0.014746409325022782 0.0001500945698485693
287 0.015625 [1.11916312 1.26144625] 0.014597695675907444 0.00014871364911533784
288 0.015625 [1.118559   1.26005248] 0.014450352069275706 0.00014734360663173733
289 0.015625 [1.11795742 1.25866525] 0.014304367688685347 0.00014598438059035962
290 0.015625 [1.11735837 1.25728455] 0.01415973177944556 0.0001446359092397865
291 0.015625 [1.11676185 1.25591036] 0.014016433648556766 0.00014329813088879463
292 0.015625 [1.11616785 1.25454265] 0.013874462664646977 0.00014197098390978866
293 0.015625 [1.11557637 1.25318141] 0.013

536 0.015625 [1.03029551 1.06390533] 0.000946533732945606 1.1030162030567438e-05
537 0.015625 [1.03012037 1.06353088] 0.0009356276933534596 1.0906039592146382e-05
538 0.015625 [1.02994619 1.06315855] 0.0009248444289689171 1.0783264384542465e-05
539 0.015625 [1.02977298 1.06278833] 0.0009141826063604173 1.066182260849979e-05
540 0.015625 [1.02960073 1.06242021] 0.0009036409057678269 1.0541700592590431e-05
541 0.015625 [1.02942942 1.06205419] 0.0008932180209755624 1.0422884792264469e-05
542 0.015625 [1.02925907 1.06169024] 0.0008829126591866367 1.0305361788925699e-05
543 0.015625 [1.02908965 1.06132837] 0.0008727235408977338 1.0189118288902945e-05
544 0.015625 [1.02892118 1.06096855] 0.0008626493997751659 1.007414112256784e-05
545 0.015625 [1.02875363 1.06061077] 0.0008526889825318543 9.960417243311671e-06
546 0.015625 [1.02858702 1.06025503] 0.0008428410488051779 9.847933726676384e-06
547 0.015625 [1.02842133 1.05990132] 0.0008331043710358472 9.736677769330666e-06
548 0.015625 [1.028256

798 0.015625 [1.00640562 1.01336665] 4.235491916573386e-05 5.123793020788656e-07
799 0.015625 [1.00636722 1.01328628] 4.184863387673633e-05 5.06285288997536e-07
800 0.015625 [1.00632904 1.01320639] 4.1348370660512616e-05 5.002632162237112e-07
801 0.015625 [1.00629109 1.01312698] 4.0854058416453904e-05 4.943122440587125e-07
802 0.015625 [1.00625337 1.01304804] 4.036562687401022e-05 4.884315424436857e-07
803 0.015625 [1.00621587 1.01296957] 3.988300658317218e-05 4.826202908380349e-07
804 0.015625 [1.00617859 1.01289157] 3.940612890504644e-05 4.768776781257405e-07
805 0.015625 [1.00614154 1.01281404] 3.89349260025496e-05 4.712029024968428e-07
806 0.015625 [1.0061047  1.01273697] 3.846933083119326e-05 4.6559517135634146e-07
807 0.015625 [1.00606808 1.01266036] 3.800927712998776e-05 4.6005370120549583e-07
808 0.015625 [1.00603168 1.01258421] 3.755469941242474e-05 4.545777175630239e-07
809 0.015625 [1.0059955  1.01250851] 3.710553295758899e-05 4.4916645483575e-07
810 0.015625 [1.00595953 1.0

1106 0.015625 [1.00099506 1.00207128] 1.0222816116114598e-06 1.2472974018429419e-08
1107 0.015625 [1.00098904 1.00205875] 1.0099588714547898e-06 1.2322740156669947e-08
1108 0.015625 [1.00098306 1.0020463 ] 9.977845577184324e-07 1.2174313736357463e-08
1109 0.015625 [1.00097712 1.00203393] 9.857568846676725e-07 1.202767305075989e-08
1110 0.015625 [1.00097121 1.00202162] 9.738740880154293e-07 1.1882796652243218e-08
1111 0.015625 [1.00096534 1.0020094 ] 9.621344246653465e-07 1.173966335008276e-08
1112 0.015625 [1.0009595  1.00199724] 9.505361724576126e-07 1.1598252207733914e-08
1113 0.015625 [1.0009537  1.00198516] 9.390776299179814e-07 1.145854253963122e-08
1114 0.015625 [1.00094794 1.00197316] 9.277571160107221e-07 1.1320513907259299e-08
1115 0.015625 [1.00094221 1.00196122] 9.165729698926605e-07 1.1184146118061546e-08
1116 0.015625 [1.00093651 1.00194936] 9.055235506721715e-07 1.1049419220489022e-08
1117 0.015625 [1.00093085 1.00193757] 8.94607237169442e-07 1.0916313502729473e-08
1118 0

...

1716 0.015625 [1.00002452 1.00005103] 6.210201377265389e-10 7.588785304376034e-12
1717 0.015625 [1.00002438 1.00005072] 6.135229651326665e-10 7.497172593872378e-12
1718 0.015625 [1.00002423 1.00005041] 6.061162993293567e-10 7.406665803309741e-12
1719 0.015625 [1.00002408 1.00005011] 5.987990477283469e-10 7.317251601009817e-12
1720 0.015625 [1.00002394 1.0000498 ] 5.915701309398468e-10 7.228916788500153e-12
1721 0.015625 [1.00002379 1.0000495 ] 5.844284826097394e-10 7.14164833010737e-12
1722 0.015625 [1.00002365 1.0000492 ] 5.773730492467265e-10 7.055433363012884e-12
1723 0.015625 [1.0000235 1.0000489] 5.704027900853744e-10 6.970259161352129e-12
1724 0.015625 [1.00002336 1.00004861] 5.63516676920553e-10 6.8861131648213594e-12
1725 0.015625 [1.00002322 1.00004831] 5.567136939470674e-10 6.802982973485645e-12
1726 0.015625 [1.00002308 1.00004802] 5.499928376377681e-10 6.720856309299252e-12
1727 0.015625 [1.00002294 1.00004773] 5.43353116575576e-10 6.63972106219212e-12
1728 0.015625 [1.0000

1941 0.015625 [1.00000625 1.00001301] 4.038700644147539e-11 4.93538240783956e-13
1942 0.015625 [1.00000622 1.00001293] 3.989942650600577e-11 4.875799354696195e-13
1943 0.015625 [1.00000618 1.00001286] 3.941773294591766e-11 4.81693560088112e-13
1944 0.015625 [1.00000614 1.00001278] 3.894185469770082e-11 4.758782482168425e-13
1945 0.015625 [1.0000061 1.0000127] 3.847172155392068e-11 4.701331437801385e-13
1946 0.015625 [1.00000607 1.00001262] 3.800726415681291e-11 4.644573971077682e-13
1947 0.015625 [1.00000603 1.00001255] 3.754841398645864e-11 4.588501703542729e-13
1948 0.015625 [1.00000599 1.00001247] 3.7095103348661995e-11 4.5331063779664217e-13
1949 0.015625 [1.00000596 1.0000124 ] 3.664726536874018e-11 4.478379799218142e-13
1950 0.015625 [1.00000592 1.00001232] 3.6204833976936576e-11 4.424313918036048e-13
1951 0.015625 [1.00000589 1.00001225] 3.576774390044143e-11 4.3709007649514365e-13
1952 0.015625 [1.00000585 1.00001217] 3.5335930658021344e-11 4.3181324242008867e-13
1953 0.015625 

2170 0.015625 [1.00000156 1.00000324] 2.5018363646146486e-12 3.057322681380451e-14
2171 0.015625 [1.00000155 1.00000322] 2.4716322414247884e-12 3.02041231898602e-14
2172 0.015625 [1.00000154 1.0000032 ] 2.4417927656308518e-12 2.983947579393658e-14
2173 0.015625 [1.00000153 1.00000318] 2.4123135348184504e-12 2.9479230812401415e-14
2174 0.015625 [1.00000152 1.00000316] 2.3831901997768807e-12 2.912333504156963e-14
2175 0.015625 [1.00000151 1.00000314] 2.354418463895334e-12 2.8771735881546694e-14
2176 0.015625 [1.0000015  1.00000312] 2.3259940825547954e-12 2.842438134053861e-14
2177 0.015625 [1.00000149 1.0000031 ] 2.2979128625470732e-12 2.8081220007722178e-14
2178 0.015625 [1.00000148 1.00000309] 2.2701706604376184e-12 2.7742202109454853e-14
2179 0.015625 [1.00000147 1.00000307] 2.242763383179428e-12 2.7407277258190373e-14
2180 0.015625 [1.00000146 1.00000305] 2.2156869880903215e-12 2.7076395089106518e-14
2181 0.015625 [1.00000146 1.00000303] 2.1889374797526954e-12 2.674950833762613e-14
2

2428 0.015625 [1.00000032 1.00000068] 1.0896690638621711e-13 1.331612393947953e-15
2429 0.015625 [1.00000032 1.00000067] 1.0765137017238972e-13 1.3155362138273969e-15
2430 0.015625 [1.00000032 1.00000067] 1.0635171626527691e-13 1.299653907112806e-15
2431 0.015625 [1.00000032 1.00000066] 1.0506775280828468e-13 1.2839634569922294e-15
2432 0.015625 [1.00000032 1.00000066] 1.0379929034442234e-13 1.2684624638623447e-15
2433 0.015625 [1.00000032 1.00000066] 1.0254614181466844e-13 1.2531485297538949e-15
2434 0.015625 [1.00000031 1.00000065] 1.0130812224832423e-13 1.2380195663442084e-15
2435 0.015625 [1.00000031 1.00000065] 1.0008504907698853e-13 1.2230731713357062e-15
2436 0.015625 [1.00000031 1.00000064] 9.887674185377387e-14 1.208307223214653e-15
2437 0.015625 [1.00000031 1.00000064] 9.768302223509174e-14 1.193719618682137e-15
2438 0.015625 [1.00000031 1.00000064] 9.650371417081038e-14 1.179308064281359e-15
2439 0.015625 [1.0000003  1.00000063] 9.533864366223901e-14 1.1650705085713687e-15
2

(2643, array([1.00000009, 1.00000018]))

In [55]:
local_descent_strong_backtracking(f, grads, x_, alpha=1)

0.04296875 [-2  2] 29
1 0.008056640625 [1.6953125 2.859375 ] 0.48454131558537483 28.515458684414625
2 1.41796875 [1.68009054 2.86056009] 0.46968847588418516 0.014852839701189668
3 0.009693145751953125 [1.55508884 2.32377576] 0.3527990036892039 0.11688947219498125
4 0.05815887451171875 [1.51583073 2.33293825] 0.2722749410924204 0.08052406259678352
5 0.010904788970947266 [1.5178864  2.31246897] 0.2685667096461876 0.0037082314462327792
6 0.1308574676513672 [1.50940203 2.31154317] 0.265017801941659 0.003548907704528592
7 0.010223239660263062 [1.50742715 2.2680348 ] 0.25757483543332377 0.007442966508335258
8 0.49071550369262695 [1.49572617 2.26847459] 0.2506359423263479 0.006938893106975874
9 0.011501144617795944 [1.46834859 2.11498945] 0.22777924840091998 0.022856693925427912
10 0.025877575390040874 [1.44370796 2.11971161] 0.20314925819195728 0.0246299902089627
11 0.019408181542530656 [1.44720857 2.11054605] 0.20129694008923835 0.0018523181027189362
12 0.019408181542530656 [1.43891256 2.10

252 0.02851635896232399 [1.00003392 1.00006835] 1.151659537097368e-09 4.846400603665191e-11
253 0.04990362818406698 [1.00003228 1.0000682 ] 1.1083388312066395e-09 4.3320705890728503e-11
254 0.031189767615041865 [1.0000327  1.00006638] 1.0739670935299668e-09 3.437173767667264e-11
255 0.0467846514225628 [1.00003127 1.00006607] 1.0402017817213755e-09 3.3765311808591296e-11
256 0.0350884885669221 [1.00003165 1.00006442] 1.0079168503886105e-09 3.228493133276504e-11
257 0.04386061070865263 [1.00003022 1.00006403] 9.776026337954185e-10 3.0314216593191975e-11
258 0.03289545803148947 [1.00003072 1.00006245] 9.487301410655566e-10 2.8872492729861925e-11
259 0.04934318704723421 [1.00002937 1.00006212] 9.196693319264826e-10 2.9060809139074028e-11
260 0.03083949190452138 [1.00002981 1.00006045] 8.920240994678311e-10 2.7645232458651426e-11
261 0.04625923785678207 [1.00002848 1.00006019] 8.633077292525762e-10 2.871637021525498e-11
262 0.03469442839258655 [1.00002883 1.0000587 ] 8.366814023750764e-10 2

(412, array([1.00000002, 1.00000004]))

In [56]:
# 문제해결과제
def f(x):
    y = x[0]**5 - 8*x[0]**3 + 10*x[0] + 6
    return y

In [57]:
x = sp.IndexedBase('x')
gradients = np.array([sp.diff(f(x), x[i]) for i in range(1)])
grads = sp.lambdify(x, gradients, 'numpy')

x_ = np.array([-2])

In [58]:
local_descent_backtracking(f, grads, x_, alpha=1)

1: alpha = 0.5000
0.5 [-2] 18
1: alpha = 0.2500
2: alpha = 0.1250
1 0.125 [1.] 9.0 9.0
1: alpha = 0.0625
2: alpha = 0.0312
3: alpha = 0.0156
2 0.015625 [2.125] -6.185028076171875 15.185028076171875
3 0.015625 [2.06907272] -6.250988262279577 0.06596018610770216
4 0.015625 [2.08638569] -6.2583239755482865 0.0073357132687092985
5 0.015625 [2.08214901] -6.258744049240125 0.0004200736918384962
6 0.015625 [2.08327727] -6.258774171722553 3.0122482428396324e-05
7 0.015625 [2.08298263] -6.2587762199384365 2.0482158831214292e-06
8 0.015625 [2.08305998] -6.258776361210465 1.4127202874192335e-07
9 0.015625 [2.0830397] -6.258776370918298 9.707832759886514e-09


(10, array([2.0830397]))

In [59]:
# alpha값의 설정에 따라 결과가 달라짐
local_descent_backtracking(f, grads, x_, alpha=0.1)

0.1 [-2] 18
1: alpha = 0.0500
1 0.05 [-1.4] 8.573759999999996 9.426240000000004
2 0.05 [-0.5084] 1.9332867673833949 6.640473232616602
3 0.05 [-0.71493709] 1.5872805237715832 0.34600624361181165
4 0.05 [-0.6668898] 1.5719454875259045 0.01533503624567878
5 0.05 [-0.68264826] 1.570230668215248 0.0017148193106564236
6 0.05 [-0.6777289] 1.5700657651478922 0.00016490306735583715
7 0.05 [-0.67929205] 1.57004904688897 1.6718258922132634e-05
8 0.05 [-0.67879801] 1.5700473790759553 1.6678130148051196e-06
9 0.05 [-0.67895442] 1.5700472118339786 1.6724197671180718e-07
10 0.05 [-0.67890493] 1.5700471950908605 1.6743118003148538e-08
11 0.05 [-0.67892059] 1.5700471934137887 1.677071814754072e-09


(12, array([-0.67892059]))

In [60]:
local_descent_strong_backtracking(f, grads, x_, alpha=1)

0.6796875 [-2] 18
1 0.013275146484375 [2.078125] -6.257803271524608 24.257803271524608
2 0.013275146484375 [2.08336606] -6.258772178536361 0.0009689070117531173
3 0.013275146484375 [2.08302044] -6.258776349378962 4.170842601070035e-06
4 0.013275146484375 [2.08304561] -6.25877637151876 2.2139797550835283e-08


(5, array([2.08304561]))